In [1]:
import pdfplumber
import os
import re
import sys
import traceback
import pickle
from tqdm import tqdm
import fitz

In [3]:
file_path = "ShareholderLetters/FINAL-Q2-23-Shareholder-Letter.pdf"

In [5]:
document = fitz.open(file_path)

In [73]:
# Define headings in the document using word size function
def word_ratio_func(word):
    try:
        # calculate word size parameters
        word_length = len(word[4])
        word_bottom = float(word[1])
        word_top = float(word[3])
        return abs(word_bottom - word_top), word_length, word[4]
        
    except:
        # in case of error, return zeros
        return 0, 0, 0

In [83]:
# Preprocess the text
def preprocess_text(texts):

    # join the text and perform cleansing operations
    text = "".join(texts.values()).strip("●").strip("*")
    text = text.split("\n")
    text = [x for x in text if x != '' and x.startswith("Source") == False]
    text = [x[0].replace("●", "") + x[1:] if x[0] == "●" else x for x in text]
    text = [x[0].replace("1", "") + x[1:] if x[1:3] in ["Q1", "Q2", "Q3", "Q4"] else x for x in text]
    text = text[2:]

    return text

In [84]:
# open the pdf file using pdfplumber library
reader = fitz.open(file_path)

# store the text and headings within one pdf file
texts = {}
headings = []
headings_count = 0

for page_number in range(len(reader)):

    # get the specific page from the pdf file
    page = reader.load_page(page_number)
    # get text from page
    text = page.get_text()
    # add text to dictionary
    texts[page_number] = text

    # get headings from page
    words = page.get_text("words")
    word_count = 0
    while word_count < len(words):
        # find if the words are large enough to be headings
        word_size, word_length, word_text = word_ratio_func(words[word_count])
        heading = []

        if word_size > 18 and word_length > 1:
            while True:
                heading.append(word_text)
                word_count += 1
                if word_count >= len(words):
                    break
                word_size, word_length, word_text = word_ratio_func(words[word_count])
                if not word_size > 18 and word_length > 1:
                    headings.append(" ".join(heading))
                    word_count += 10
                    break
        headings_count += 1
        word_count += 1
    
    # break if the page covers the reference section
    if re.search("\nReference\n", text):
        break
        
    
# preprocess the text
text = preprocess_text(texts)
final_text = " ".join(text).strip()

In [145]:
import pickle

# load the dictionary with the pdf paths
with open("pdf_texts.pkl", "rb") as f:
    pdf_texts = pickle.load(f)

# load the dictionary with the pdf texts
with open("pdf_headings.pkl", "rb") as f:
    pdf_headings = pickle.load(f)

In [ ]:
pdf_texts

In [112]:
for index, value in pdf_headings.items():
    value_2 = value
    break

In [138]:
# remove item that contain "\u" from the list
# [x for x in value_2 if not re.search("200b", x)]
a = [x.replace("\u200b", "remove") for x in value_2]
[x for x in a if not re.search("remove", x)]

['Q4 Results',
 'Forecast',
 'Content',
 'Product and Partnerships',
 'Competition',
 'Free Cash Flow and Capital Structure',
 'Board of Directors',
 'Summary',
 '22, 2018 Earnings Interview, 3pm PST']

In [25]:
texts

{0: "July 19, 2023\nSummary:\n●\nIn May, we successfully launched paid sharing in 100+ countries, representing more than 80% of\nour revenue base.\n○\nRevenue in each region is now higher than pre-launch, with sign-ups already exceeding\ncancellations.\n○\nPaid net additions were 5.9M in Q2, and today we’re rolling out paid sharing to almost\nall of the remaining countries.\n●\nQ2‘23 revenue of $8.2B and operating profit of $1.8B were generally in-line with our\nforecast—and we expect revenue growth to accelerate in the second half of ‘23 as we start to\nsee the full benefits of paid sharing plus continued steady growth in our ad-supported plan.\nWe’re still targeting a full year 2023 operating margin of 18% to 20%.\n●\nWe’re a leader in terms of streaming engagement and, per Nielsen, we had the top original\nstreaming series in the US for 24 of the first 25 weeks of 2023, and the top movie for 21 weeks.\nWhile we’ve made steady progress this year, we have more work to do to reaccelera

In [26]:
text = preprocess_text(texts)
final_text = " ".join(text)

In [28]:
final_text

' In May, we successfully launched paid sharing in 100+ countries, representing more than 80% of our revenue base. ○ Revenue in each region is now higher than pre-launch, with sign-ups already exceeding cancellations. ○ Paid net additions were 5.9M in Q2, and today we’re rolling out paid sharing to almost all of the remaining countries.  Q2‘23 revenue of $8.2B and operating profit of $1.8B were generally in-line with our forecast—and we expect revenue growth to accelerate in the second half of ‘23 as we start to see the full benefits of paid sharing plus continued steady growth in our ad-supported plan. We’re still targeting a full year 2023 operating margin of 18% to 20%.  We’re a leader in terms of streaming engagement and, per Nielsen, we had the top original streaming series in the US for 24 of the first 25 weeks of 2023, and the top movie for 21 weeks. While we’ve made steady progress this year, we have more work to do to reaccelerate our growth. We remain focused on: creating a s

In [6]:
# Define headings in the document using word size function
def word_ratio_func(word):
    try:
        # calculate word size parameters
        word_length = len(word["text"])
        word_bottom = float(word['bottom'])
        word_top = float(word['top'])
        return (word_bottom - word_top), word_length, word["text"]
        
    except:
        # in case of error, return zeros
        return 0, 0, 0

# Preprocess the text
def preprocess_text(texts):

    # join the text and perform cleansing operations
    text = "".join(texts.values()).strip("●").strip("*")
    text = text.split("\n")
    text = [x for x in text if x != '' and x.startswith("Source") == False]
    text = [x[0].replace("●", "") + x[1:] if x[0] == "●" else x for x in text]
    text = [x[0].replace("*", "") + x[1:] if x[0] == "*" else x for x in text]
    text = [x[0].replace("○", "") + x[1:] if x[0] == "○" else x for x in text]
    text = [x[0].replace("1", "") + x[1:] if x[1:3] in ["Q1", "Q2", "Q3", "Q4"] else x for x in text]
    text = text[2:]

    return text

In [12]:
# Transform pdf into the text
def process_pdf(pdf_paths):
    
    # store the pdf text and headings in dictionaries
    pdf_texts = {}
    pdf_headings = {}

    # iterate over the pdf files
    for file_path in tqdm(pdf_paths):

        # if file path contains
        file_path = "ShareholderLetters/FINAL-Q2-23-Shareholder-Letter.pdf"

        try:
            
            # open the pdf file using pdfplumber library
            reader = pdfplumber.open(file_path)

            # store the text and headings within one pdf file
            texts = {}
            headings = []
            headings_count = 0

            for page_number in range(0, len(reader.pages)):

                # get the specific page from the pdf file
                page = reader.pages[page_number]
                # extract text from page
                text = page.extract_text()
                # add text to dictionary
                texts[page_number] = text

                # extract headings from page
                words = page.extract_words()
                word_count = 0
                while word_count < len(words):
                    # find if the words are large enough to be headings
                    word_size, word_length, word_text = word_ratio_func(words[word_count])
                    heading = []

                    if word_size > 15 and word_length > 1:
                        while True:
                            heading.append(word_text)
                            word_count += 1
                            if word_count >= len(words):
                                break
                            word_size, word_length, word_text = word_ratio_func(words[word_count])
                            if not word_size > 15 and word_length > 1:
                                headings.append(" ".join(heading))
                                word_count += 10
                                break
                    headings_count += 1
                    word_count += 1
                
                # break if the page covers the reference section
                if re.search("\nReference\n", text):
                    break
            
            # preprocess the text
            text = preprocess_text(texts)
            final_text = " ".join(text)

            # optionally, export the text to a txt file
            # with open("Txt/" + file_path.split("/")[-1].split(".")[0] + ".txt", "w", encoding='utf-8') as f:
            #     f.write(final_text)

            # add the text to the dictionary
            pdf_texts[file_path.split("/")[-1].split(".")[0]] = final_text
            # add the headings to the dictionary
            pdf_headings[file_path.split("/")[-1].split(".")[0]] = headings

        except Exception as e:

            # in case of error, print the specifics of issue
            exc_type, exc_value, exc_traceback = sys.exc_info()
            traceback_details = traceback.extract_tb(exc_traceback)
            filename = traceback_details[-1][0]
            line_no = traceback_details[-1][1]
            func = traceback_details[-1][2]
            print(f"Exception occurred in file {filename} at line {line_no} in function {func}")
            print(f"Exception type: {exc_type.__name__}, Exception message: {str(e)}")

            break
    
    return pdf_texts, pdf_headings

# Transform pdf files into texts and headings and store them as dictionaries
pdf_texts, pdf_headings = process_pdf(file_paths) # total run time: 2 min 20 s 20 files

  0%|          | 0/50 [00:00<?, ?it/s]

In [11]:
# Get file paths for the pdf files
folder_path = "ShareholderLetters/" # put '/' sign at the end of the folder
file_paths = []
for root, directories, files in os.walk(folder_path):
    for filename in files:
        filepath = os.path.join(root, filename)
        file_paths.append(filepath)

In [9]:
file_paths

['ShareholderLetters/COMBINED-Q4-17-Shareholder-Letter-FINAL.pdf',
 'ShareholderLetters/FINAL-Q1-18-Shareholder-Letter.pdf',
 'ShareholderLetters/FINAL-Q1-19-Shareholder-Letter.pdf',
 'ShareholderLetters/FINAL-Q1-20-Shareholder-Letter.pdf',
 'ShareholderLetters/FINAL-Q1-21-Shareholder-Letter.pdf',
 'ShareholderLetters/FINAL-Q1-22-Shareholder-Letter.pdf',
 'ShareholderLetters/Final-Q1-23-Shareholder-Letter.pdf',
 'ShareholderLetters/FINAL-Q2-18-Shareholder-Letter.pdf',
 'ShareholderLetters/FINAL-Q2-20-Shareholder-Letter-V3-with-Tables.pdf',
 'ShareholderLetters/FINAL-Q2-21-Shareholder-Letter.pdf',
 'ShareholderLetters/FINAL-Q2-22-Shareholder-Letter.pdf',
 'ShareholderLetters/FINAL-Q2-23-Shareholder-Letter.pdf',
 'ShareholderLetters/FINAL-Q3-18-Shareholder-Letter.pdf',
 'ShareholderLetters/FINAL-Q3-19-Shareholder-Letter.pdf',
 'ShareholderLetters/FINAL-Q3-20-Shareholder-Letter.pdf',
 'ShareholderLetters/FINAL-Q3-21-Shareholder-Letter.pdf',
 'ShareholderLetters/FINAL-Q3-22-Shareholder-Let

In [ ]:
# Transform pdf files into texts and headings and store them as dictionaries
pdf_texts, pdf_headings = process_pdf(file_paths) # total run time: 2 min 20 s 20 files

# Save pdf texts and headings to pickle files
with open("pdf_texts.pkl", "wb") as f:
    pickle.dump(pdf_texts, f)
with open("pdf_headings.pkl", "wb") as f:
    pickle.dump(pdf_headings, f)
